# Выбор локации для скважины

Условной добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Также нужно проанализировать возможную прибыль и риски техникой *Bootstrap.*

**Шаги для выбора локации:**

1. В избранном регионе ищут месторождения, для каждого определяют значения признаков;
2. Строят модель и оценивают объём запасов;
3. Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
4. Прибыль равна суммарной прибыли отобранных месторождений.


**Описание данных**

Данные геологоразведки трёх регионов находятся в трёх файлах.

- `id` — уникальный идентификатор скважины;
- `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
- `product` — объём запасов в скважине (тыс. баррелей).

**Условия задачи:**

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

**Содержание**<a id='toc0_'></a>    
- 1. [Загрузка и подготовка данных](#toc1_)    
- 2. [Обучение и проверка модели](#toc2_)    
- 3. [Подготовка к расчёту прибыли](#toc3_)    
- 4. [Расчёт прибыли и рисков](#toc4_)    
- 5. [Общий вывод](#toc5_)    
- 6. [Чек-лист готовности проекта](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Загрузка и подготовка данных](#toc0_)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Импортируем данные и изучим датасеты.

In [2]:
raw_data_0 = pd.read_csv('geo_data_0.csv')
raw_data_1 = pd.read_csv('geo_data_1.csv')
raw_data_2 = pd.read_csv('geo_data_2.csv')
datasets = [raw_data_0, raw_data_1, raw_data_2]

In [3]:
for number, dataset in enumerate(datasets):
    print(f'Регион №{number+1}')
    print(dataset.head(), '\n')
    print(dataset.info())
    print(dataset.describe(), '\n')
    print(dataset.drop(columns=['id']).corr())

Регион №1
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.2

Пропусков в данных нет, все столбцы имеют правильный тип данных.

Отделим целевой признак от остальных, считая, что идентификатор скважины не несёт информации о запасах нефти в ней.

In [4]:
features = [df[['f0', 'f1', 'f2']] for df in datasets]
target = [df['product'] for df in datasets]
print(features[0].shape, target[0].shape)

(100000, 3) (100000,)


**Вывод по разделу 1:**
- Данные импортированы, изучены, пропусков в них нет, все столбцы имеют правильный тип данных.
- Целевой признак отделён от остальных.

## 2. <a id='toc2_'></a>[Обучение и проверка модели](#toc0_)

Напишем функцию, строящую и проверяющую модель для региона.

In [5]:
state = np.random.RandomState(245234)

In [6]:
def predict_product(features, target):
    #Разбиение данных на выборки
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=state)
    
    #Масштабирование данных
    scaler = StandardScaler().fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    #Обучение модели
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    #Предсказания
    predictions_valid = model.predict(features_valid)
    print('Средний запас предсказанного сырья:', predictions_valid.mean())
    print('RMSE:', mean_squared_error(target_valid, predictions_valid) ** 0.5)
    
    return pd.Series(predictions_valid), target_valid.reset_index(drop=True)

Построим модели для каждого региона, найдём средний запас предсказанного сырья, и величину RMSE.

Регион №1:

In [7]:
predictions_reg_1, target_reg_1 = predict_product(features[0], target[0])

Средний запас предсказанного сырья: 92.6029292151203
RMSE: 37.66437187122903


Регион №2:

In [8]:
predictions_reg_2, target_reg_2 = predict_product(features[1], target[1])

Средний запас предсказанного сырья: 68.75490010763494
RMSE: 0.888452042827013


Регион №3:

In [9]:
predictions_reg_3, target_reg_3 = predict_product(features[2], target[2])

Средний запас предсказанного сырья: 94.689828875853
RMSE: 40.0864160481026


**Вывод по разделу 2:**
- Для каждого региона построена модель линейной регрессии, получены предсказания на валидационнной выборке.
- Средние запасы предсказанного сырья в регионах 1 и 3 в 1.4 раза больше, чем в регионе 2; однако во втором регионе точность предсказаний значительно больше (примерно в 45 раз).

## 3. <a id='toc3_'></a>[Подготовка к расчёту прибыли](#toc0_)

Сохраним все необходимые параметры в отдельных переменных.

In [10]:
RESEARCHED_WELLS_SUM = 500
BEST_WELLS_NUM = 200
REGION_BUDGET = 10_000_000_000
THOUSAND_BARREL_COST = 450_000
ACCEPTABLE_RISK = 0.025

Найдём достаточный объём сырья для безубыточной разработки новой скважины.

In [11]:
sufficient_volume = REGION_BUDGET / BEST_WELLS_NUM / THOUSAND_BARREL_COST
print(f'Достаточный объём сырья для безубыточной разработки новой скважины: {sufficient_volume:.1f} тыс. баррелей')

Достаточный объём сырья для безубыточной разработки новой скважины: 111.1 тыс. баррелей


Средний запас сырья по регионам:

In [12]:
for num, stocks in enumerate(target):
    print(f'Регион №{num+1}. Средний запас: {stocks.mean():.2f}')

Регион №1. Средний запас: 92.50
Регион №2. Средний запас: 68.83
Регион №3. Средний запас: 95.00


Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.

In [13]:
def calculate_revenue(target, predictions, sample_size=BEST_WELLS_NUM):
    best_wells_indexes = pd.Series(predictions).nlargest(sample_size).index
    volume = target[best_wells_indexes].sum()
    revenue = volume * THOUSAND_BARREL_COST - REGION_BUDGET
    return revenue

**Вывод по разделу 3:**
- Для безубыточной разработки новой скважины необходимо добыть из неё 111 тыс. баррелей сырья.
- Этот показатель превышает средние запасы в скважинах регионов 1 и 3 на 20 тыс. баррелей, региона 2 - на 42 тыс.
- Написана функция для расчёта общей выручки в регионе после разработки 200 скважин с наибольшими залежами сырья.

## 4. <a id='toc4_'></a>[Расчёт прибыли и рисков](#toc0_)

Для определения рисков найдём распределение прибыли, используя технику бутстреп.

In [14]:
def calculate_risk(target, predictions, interval_width, n_samples=1000, sample_size=RESEARCHED_WELLS_SUM):
    distribution = []
    for i in range(n_samples):
        sample = predictions.sample(n=sample_size, replace=True, random_state=state)
        distribution.append(calculate_revenue(target, sample))
    distribution = pd.Series(distribution)
    print(f'Средняя прибыль: {distribution.mean()/10e6:.2f} млн. руб')
    print(f'Доверительный интервал: '
          f'({distribution.quantile(0.5-interval_width/2)/10e6:.2f}; '
          f'{distribution.quantile(0.5+interval_width/2)/10e6:.2f}) млн. руб')
    risk = distribution[distribution < 0].count() / distribution.count()
    print(f'Риск убытков: {risk:.1%}')
    return risk

<div class="alert alert-block alert-success">
<b>Успех:</b> С бутстрапом тоже все в порядке, метрики считаются верным образом!  
</div>

Регион №1:

In [15]:
calculate_risk(target_reg_1, predictions_reg_1, interval_width=0.95);

Средняя прибыль: 37.97 млн. руб
Доверительный интервал: (-13.47; 92.76) млн. руб
Риск убытков: 6.5%


Регион №2:

In [16]:
calculate_risk(target_reg_2, predictions_reg_2, interval_width=0.95);

Средняя прибыль: 44.12 млн. руб
Доверительный интервал: (3.16; 84.25) млн. руб
Риск убытков: 1.6%


Регион №3:

In [17]:
calculate_risk(target_reg_3, predictions_reg_3, interval_width=0.95);

Средняя прибыль: 42.01 млн. руб
Доверительный интервал: (-10.25; 94.32) млн. руб
Риск убытков: 6.1%


**Вывод по разделу 4:**
- Для каждого региона определены средняя прибыль, 95%-й доверительный интервал и риск убытков.
- Единственный регион, где риск убытков ниже 2.5% - регион №2. Средняя прибыль составляет 44.12 млн. руб.

## 5. <a id='toc5_'></a>[Общий вывод](#toc0_)

- На основании исходных данных для каждого из исследуемых регионов построена модель линейной регрессии, получены предсказания на валидационной выборке.
- Для безубыточной разработки новой скважины необходимо добыть из неё 111 тыс. баррелей сырья.
- Для каждого региона определены средняя прибыль, 95%-й доверительный интервал и риск убытков.
- Единственный регион, где риск убытков ниже 2.5% - регион №2. Средняя прибыль составляет 44.12 млн. руб.

## 6. <a id='toc6_'></a>[Чек-лист готовности проекта](#toc0_)

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные подготовлены
- [x]  Модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован